# Analysis preview

This preview helps me know what kind of data I'm moving and how should I transform it

## Variables, libraries, configs

In [80]:
import os
import requests
import sqlite3
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)  

In [81]:
#loading environment variables
load_dotenv() 

STATION_ID = os.getenv("STATION_ID", "KGPH")
API_BASE_URL = os.getenv("API_BASE_URL", "https://api.weather.gov")
USER_AGENT = os.getenv("USER_AGENT", "(example_weather_app, contact@example.com)")

In [82]:
# url for each endpoint
url_station = f"{API_BASE_URL}/stations/{STATION_ID}"
url_obs = f"{API_BASE_URL}/stations/{STATION_ID}/observations"

#extraction start date (last 7 days)
start_date = (datetime.now() - timedelta(days=7))
#start date format should be YYYY-MM-DDThh:mm:ssZ or YYYY-MM-DDThh:mm:ss+hh:mm
start_date_str = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

## Extraction

### Stations endpoint

In [83]:
headers = {"User-Agent": USER_AGENT}
station = requests.get(url_station, headers=headers).json()

In [84]:
station #should handle errors (if station not found, etc)

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'id': 'https://api.weather.gov/stations/KGPH',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-94.3096, 39.3325]},
 'properties': {'@id': 'https://api.weather.gov/stations/KG

In [85]:
# saving station  timezone
station_timezone = station['properties']['timeZone']

### Observations endpoint

In [86]:
observations = requests.get(
    url_obs,
    headers=headers,
    params={"start": start_date_str}
).json()

In [87]:
observations

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'type': 'FeatureCollection',
 'features': [{'id': 'https://api.weather.gov/stations/KGPH/observations/2025-07-07T19:55:00+00:00',
   'type': 'Feature',
   'geometry': {'type': 'Point', 'coordina

In [88]:
# loading the data into a pandas dataframe
df = pd.json_normalize(observations['features'])

In [89]:
df.head()

,id,type,geometry.type,geometry.coordinates,properties.@id,properties.@type,properties.elevation.unitCode,properties.elevation.value,properties.station,properties.stationId,properties.stationName,properties.timestamp,properties.rawMessage,properties.textDescription,properties.icon,properties.presentWeather,properties.temperature.unitCode,properties.temperature.value,properties.temperature.qualityControl,properties.dewpoint.unitCode,properties.dewpoint.value,properties.dewpoint.qualityControl,properties.windDirection.unitCode,properties.windDirection.value,properties.windDirection.qualityControl,properties.windSpeed.unitCode,properties.windSpeed.value,properties.windSpeed.qualityControl,properties.windGust.unitCode,properties.windGust.value,properties.windGust.qualityControl,properties.barometricPressure.unitCode,properties.barometricPressure.value,properties.barometricPressure.qualityControl,properties.seaLevelPressure.unitCode,properties.seaLevelPressure.value,properties.seaLevelPressure.qualityControl,properties.visibility.unitCode,properties.visibility.value,properties.visibility.qualityControl,properties.maxTemperatureLast24Hours.unitCode,properties.maxTemperatureLast24Hours.value,properties.minTemperatureLast24Hours.unitCode,properties.minTemperatureLast24Hours.value,properties.precipitationLastHour.unitCode,properties.precipitationLastHour.value,properties.precipitationLastHour.qualityControl,properties.precipitationLast3Hours.unitCode,properties.precipitationLast3Hours.value,properties.precipitationLast3Hours.qualityControl,properties.precipitationLast6Hours.unitCode,properties.precipitationLast6Hours.value,properties.precipitationLast6Hours.qualityControl,properties.relativeHumidity.unitCode,properties.relativeHumidity.value,properties.relativeHumidity.qualityControl,properties.windChill.unitCode,properties.windChill.value,properties.windChill.qualityControl,properties.heatIndex.unitCode,properties.heatIndex.value,properties.heatIndex.qualityControl,properties.cloudLayers
0,https://api.weather.gov/stations/KGPH/observat...,Feature,Point,"[-94.31, 39.33]",https://api.weather.gov/stations/KGPH/observat...,wx:ObservationStation,wmoUnit:m,237,https://api.weather.gov/stations/KGPH,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T19:55:00+00:00,KGPH 071955Z AUTO 17004KT 10SM CLR 31/19 A3003...,Clear,https://api.weather.gov/icons/land/day/skc?siz...,[],wmoUnit:degC,31.0,V,wmoUnit:degC,19.0,V,wmoUnit:degree_(angle),170.0,V,wmoUnit:km_h-1,7.56,V,wmoUnit:km_h-1,None,Z,wmoUnit:Pa,101690.0,V,wmoUnit:Pa,None,Z,wmoUnit:m,16090,C,wmoUnit:degC,None,wmoUnit:degC,None,wmoUnit:mm,NaN,Z,wmoUnit:mm,None,Z,wmoUnit:mm,None,Z,wmoUnit:percent,48.878892,V,wmoUnit:degC,None,V,wmoUnit:degC,32.399899,V,"[{'base': {'unitCode': 'wmoUnit:m', 'value': N..."
1,https://api.weather.gov/stations/KGPH/observat...,Feature,Point,"[-94.31, 39.33]",https://api.weather.gov/stations/KGPH/observat...,wx:ObservationStation,wmoUnit:m,237,https://api.weather.gov/stations/KGPH,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T18:55:00+00:00,KGPH 071855Z AUTO 17006KT 10SM SCT037 SCT080 3...,Partly Cloudy,https://api.weather.gov/icons/land/day/sct?siz...,[],wmoUnit:degC,31.0,V,wmoUnit:degC,21.0,V,wmoUnit:degree_(angle),170.0,V,wmoUnit:km_h-1,11.16,V,wmoUnit:km_h-1,None,Z,wmoUnit:Pa,101730.0,V,wmoUnit:Pa,None,Z,wmoUnit:m,16090,C,wmoUnit:degC,None,wmoUnit:degC,None,wmoUnit:mm,NaN,Z,wmoUnit:mm,None,Z,wmoUnit:mm,None,Z,wmoUnit:percent,55.321861,V,wmoUnit:degC,None,V,wmoUnit:degC,33.737806,V,"[{'base': {'unitCode': 'wmoUnit:m', 'value': 1..."
2,https://api.weather.gov/stations/KGPH/observat...,Feature,Point,"[-94.31, 39.33]",https://api.weather.gov/stations/KGPH/observat...,wx:ObservationStation,wmoUnit:m,237,https://api.weather.gov/stations/KGPH,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T17:55:00+00:00,KGPH 071755Z AUTO 00000KT 10SM SCT039 30/21 A3...,Partly Cloudy,https://api.weather.gov/icons/land/day/sct?siz...,[],wmoUnit:degC,30.0,V,wmoU

In [90]:
df.columns

Index(['id', 'type', 'geometry.type', 'geometry.coordinates', 'properties.@id',
       'properties.@type', 'properties.elevation.unitCode',
       'properties.elevation.value', 'properties.station',
       'properties.stationId', 'properties.stationName',
       'properties.timestamp', 'properties.rawMessage',
       'properties.textDescription', 'properties.icon',
       'properties.presentWeather', 'properties.temperature.unitCode',
       'properties.temperature.value', 'properties.temperature.qualityControl',
       'properties.dewpoint.unitCode', 'properties.dewpoint.value',
       'properties.dewpoint.qualityControl',
       'properties.windDirection.unitCode', 'properties.windDirection.value',
       'properties.windDirection.qualityControl',
       'properties.windSpeed.unitCode', 'properties.windSpeed.value',
       'properties.windSpeed.qualityControl', 'properties.windGust.unitCode',
       'properties.windGust.value', 'properties.windGust.qualityControl',
       'properties

In [91]:
# mapping the fields to the database schema
# station_id = properties.stationId
# station_name = properties.stationName
# station_timezone = missing - taking it from station metadata in station_timezone
# latitude/longitude = geometry.coordinates	
# observation timestamp = properties.timestamp
# temperature = properties.temperature.value
    # temperature unit = properties.temperature.unitCode
#wind speed = properties.windSpeed.value
    # wind speed unit = properties.windSpeed.unitCode
#humidity = properties.relativeHumidity.value

## Cleaning

In [92]:
columns_needed = [
    'properties.stationId',
    'properties.stationName',
    'properties.timestamp',
    'properties.temperature.value',
    # 'properties.temperature.unitCode',
    'properties.windSpeed.value',
    # 'properties.windSpeed.unitCode',
    'properties.relativeHumidity.value',
    'geometry.coordinates'
]
#filtering columns
df_filtered = df[columns_needed]
#renaming columns to match the database schema
df_filtered.columns = [
    'station_id',
    'station_name',
    'timestamp',
    # 'temperature_value',
    'temperature_unit',
    # 'wind_speed_value',
    'wind_speed_unit',
    'humidity_value',
    'coordinates'
]

In [93]:
df_filtered.head()

,station_id,station_name,timestamp,temperature_unit,wind_speed_unit,humidity_value,coordinates
0,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T19:55:00+00:00,31.0,7.56,48.878892,"[-94.31, 39.33]"
1,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T18:55:00+00:00,31.0,11.16,55.321861,"[-94.31, 39.33]"
2,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T17:55:00+00:00,30.0,0.00,58.581385,"[-94.31, 39.33]"
3,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T15:35:00+00:00,28.0,5.40,65.771195,"[-94.31, 39.33]"
4,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T14:55:00+00:00,28.0,0.00,65.771195,"[-94.31, 39.33]"


In [94]:
df_filtered['latitude'] = df_filtered['coordinates'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df_filtered['longitude'] = df_filtered['coordinates'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
df_filtered.drop(columns=['coordinates'], inplace=True)

/var/folders/xl/dpqgd5313dvdp7z5kg6whtn40000gn/T/ipykernel_40956/997662286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['latitude'] = df_filtered['coordinates'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
/var/folders/xl/dpqgd5313dvdp7z5kg6whtn40000gn/T/ipykernel_40956/997662286.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['longitude'] = df_filtered['coordinates'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
/var/folder

In [95]:
df_filtered

,station_id,station_name,timestamp,temperature_unit,wind_speed_unit,humidity_value,latitude,longitude
0,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T19:55:00+00:00,31.0,7.56,48.878892,-94.31,39.33
1,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T18:55:00+00:00,31.0,11.16,55.321861,-94.31,39.33
2,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T17:55:00+00:00,30.0,0.00,58.581385,-94.31,39.33
3,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T15:35:00+00:00,28.0,5.40,65.771195,-94.31,39.33
4,KGPH,Mosby Midwest National Air Center Airport,2025-07-07T14:55:00+00:00,28.0,0.00,65.771195,-94.31,39.33
...,...,...,...,...,...,...,...,...
276,KGPH,Mosby Midwest National Air Center Airport,2025-06-30T22:35:00+00:00,24.0,5.40,78.354092,-94.31,39.33
277,KGPH,Mosby Midwest National Air Center Airport,2025-06-30T22:15:00+00:00,23.0,5.40,78.209170,-94.31,39.33
278,KGPH,Mosby Midwest National Air Center Airport,2025-06-30T21:55:00+00:00,23.0,9.36,78.209170,-94.31,39.33
279,KGPH,Mosby Midwest National Air Center Airport,2025-06-30T21:35:00+00:00,22.0,7.56,83.107688,-94.31,39.33


## QA

In [96]:
# null values
df_filtered.isnull().sum()

station_id          0
station_name        0
timestamp           0
temperature_unit    1
wind_speed_unit     2
humidity_value      4
latitude            0
longitude           0
dtype: int64

In [ ]:
# max and min timestamps (to check the range)
max_timestamp = df_filtered['timestamp'].max()
min_timestamp = df_filtered['timestamp'].min()
print(f"Max timestamp: {max_timestamp}")
print(f"Min timestamp: {min_timestamp}") 

Max timestamp: 2025-07-07T19:55:00+00:00
Min timestamp: 2025-06-30T21:15:00+00:00
